### Riddler Express

On the Food Network’s latest game show, Cranberries or Bust, you have a choice between two doors: A and B. One door has a lifetime supply of cranberry sauce behind it, while the other door has absolutely nothing behind it. And boy, do you love cranberry sauce.

Of course, there’s a twist. The host presents you with a coin with two sides, marked A and B, which correspond to each door. The host tells you that the coin is weighted in favor of the cranberry door — without telling you which door that is — and that door’s letter will turn up 60 percent of the time. For example, if the sauce is behind door A, then the coin will turn up A 60 percent of the time and B the remaining 40 percent of the time.

You can flip the coin twice, after which you must make your selection. Assuming you optimize your strategy, what are your chances of choosing the door with the cranberry sauce?

### Analytical Solution: 

We have `2^2` possibilities:`{T-T,T-H,H-T,HH}`

Breaking it down: assume `h` indicates the cranberry 
- `T-T`: 0.16
- `T-H`: 0.24
- `H-T`: 0.24
- `H-H`: 0.36

Optimizing strategy makes me think we should be consdidering conditional probability. However, we assume each coin flip is independent of the next (proven below):

What is the likelihood of a heads if we already flipped a heads? 

- `P(H|H) = P(H & H) / P(H) = 0.36 / 0.6 = 0.6`
  
What is the likelihood of a tails if we already flipped a tails? 

- `P(T|T) = P(T & T) / P(T) = 0.16 / 0.4 = 0.4`

#### Using Conditional Probability: 

We can determine likelihood that a double flip would indicate a win. 

- `W` = win
- `D` = double-flip

- P(W|D) = likelihood that a double flip is a win
- P(W and D) = likelihood of win with two flips = flipping two heads = 0.6 * 0.6 = 0.36
- P(D) = Likelihood of any double flip (accounting for TT and HH)

$P(W | D) = \frac{P(W and D)}{P(D)} = \frac{0.36}{0.16 + 0.36} = 0.692 $

The above shows that if we flip a double, we have a likelihood of .692 of it being the cranberry side. 

What about the other? Well if if we flip each side in a sequence we have a 50% chance of choosing the proper side. 

Finally, we can solve for a win: 

$P(W) = P(W | D) * P(D) + P(W | S) * P(S)$

$P(W) = 0.692 * 0.52 + 0.5 * 0.48 = 0.599$

#### Extra credit: Instead of two flips, what if you are allowed three or four flips? Now what are your chances of choosing the door with the cranberry sauce?

We have `2^3` possibilities:`{T-T-T,T-H-H,T-T-H, T-H-T, H-T-T, H-H-T, H-T-H, H-H-H}`

Breaking it down: assume `h` indicates the cranberry 
- `T-T-T`
- `T-H-H`
- `T-T-H`
- `T-H-T`
- `H-T-T`
- `H-H-T`
- `H-T-H`
- `H-H-H`

Just going to solve this analytically in Python

In [1]:
# General Probs
p_ttt = 0.4**3
p_hhh = 0.6**3
p_tth = (0.4**2) * 0.6 # 3 of these
p_hht = (0.6**2) * 0.4 # 3 of these

print(3*p_hht + 3*p_tth + p_ttt + p_hhh)

# Conditionals: 
p_win_two = 3*p_hht / (3*p_hht + 3*p_tth)
p_win_three = p_hhh / (p_hhh + p_ttt)

# Solving: Prob win 2 * likelihood of 2 + prob win 3 * likelihood of 3
s = round(p_win_two * (3*p_hht + 3*p_tth) + p_win_three * (p_hhh + p_ttt),5)
print(f"Solution with 3 coins: {s}")

1.0
Solution with 3 coins: 0.648


### Solving with Numpy:

The strategy will be to choose the door based on which coin flips the most.

Two-sided:
- If we flip the same side of coins, then go for it.
- If we flip different coins, then we guess.

Three-sided:
- If we flip 0-0-1, then we would choose door 0. 
- If we flip 1-0-1, then we choose door 1. 

In [2]:
import numpy as np 

In [3]:
# 2 coins
# door is always 1

np.random.seed(42)

p = 0.6 # links to 1
coins = 2

for n in [10_000, 100_000, 1_000_000, 10_000_000]:

    # simulate n games
    arr = np.random.binomial(1, p, (n,coins))

    # if coins are the same then choose door 1, else guess randomly (50%)
    # annoyingly, np.where won't 
    w_choice = np.sum((arr[:,0] == 1) & (arr[:,1] == 1))
    l_choice = np.sum((arr[:,0] == 0) & (arr[:,1] == 0))
    
    # np.random.choice(2, replace = True)
    necessary_size = n - np.sum(w_choice) - np.sum(l_choice)
    
    # randomly guess on H-T, T-H
    random_choice = np.random.choice(2, necessary_size)

    print(f"For {n} games and flipping {coins} coins the likelihood was: {(np.sum(w_choice) + np.sum(random_choice)) / n:.4f}")

For 10000 games and flipping 2 coins the likelihood was: 0.5999
For 100000 games and flipping 2 coins the likelihood was: 0.5980
For 1000000 games and flipping 2 coins the likelihood was: 0.6001
For 10000000 games and flipping 2 coins the likelihood was: 0.5999


In [4]:
# 3 coins:
# door is always 1
np.random.seed(42)
p = 0.6 # links to 1
coins = 3

for n in [10_000, 100_000, 1_000_000, 10_000_000]:

    # simulate n games
    arr = np.random.binomial(1, p, (n,coins))

    # strategy: sum across coins
    sum_coins = np.sum(arr, axis = 1)

    # if coins >= 2 then choose door 1, else 0
    choice = np.where(sum_coins >= (coins-1), 1, 0)

    print(f"For {n} games and flipping {coins} coins the likelihood was: {np.sum(choice) / n:.4f}")

For 10000 games and flipping 3 coins the likelihood was: 0.6504
For 100000 games and flipping 3 coins the likelihood was: 0.6461
For 1000000 games and flipping 3 coins the likelihood was: 0.6484
For 10000000 games and flipping 3 coins the likelihood was: 0.6480


### Flip 4 coins

Logic: 
- 3 >= of same coins means we guess 1
    - We can back into this by finding a sum >= 3, which would be a correct guess (w_choice)
    - Or a sum <= 1, which would be an incorrect guess (l_choice)

- otherwise we flip to determine which door to choose

In [5]:
# 2 coins
# door is always 1

np.random.seed(42)

p = 0.6 # links to 1
coins = 4

for n in [10_000, 100_000, 1_000_000, 10_000_000]:

    # simulate n games
    arr = np.random.binomial(1, p, (n,coins))

    # if coins are the same then choose door 1
    # we are correct when all flips are 1 (w_choice), otherwise we lost (l_choice)
    # annoyingly, np.where won't 
    w_choice = np.sum(np.sum(arr, axis = 1) >= 3)
    l_choice = np.sum(np.sum(arr, axis = 1) <= 1)
    
    # guess randomly (50%)
    # np.random.choice(2, replace = True)
    necessary_size = n - w_choice - l_choice
    
    # randomly guess on H-T, T-H
    random_choice = np.random.choice(2, necessary_size)

    print(f"For {n} games and flipping {coins} coins the likelihood was: {(w_choice + np.sum(random_choice)) / n:.4f}")

For 10000 games and flipping 4 coins the likelihood was: 0.6447
For 100000 games and flipping 4 coins the likelihood was: 0.6469
For 1000000 games and flipping 4 coins the likelihood was: 0.6479
For 10000000 games and flipping 4 coins the likelihood was: 0.6479


### 5 Flips


In [8]:
# 5 coins:
# door is always 1
np.random.seed(42)
p = 0.6 # links to 1
coins = 5

for n in [10_000, 100_000, 1_000_000, 10_000_000]:

    # simulate n games
    arr = np.random.binomial(1, p, (n,coins))

    # strategy: sum across coins
    sum_coins = np.sum(arr, axis = 1)

    # if coins >= 2 then choose door 1, else 0
    choice = np.where(sum_coins >= 3, 1, 0)

    print(f"For {n} games and flipping {coins} coins the likelihood was: {np.sum(choice) / n:.4f}")

For 10000 games and flipping 5 coins the likelihood was: 0.6826
For 100000 games and flipping 5 coins the likelihood was: 0.6799
For 1000000 games and flipping 5 coins the likelihood was: 0.6827
For 10000000 games and flipping 5 coins the likelihood was: 0.6826
